In [0]:
from delta.tables import *

stage_table = 'incremental_load.default.orders_stage'
target_table_name = 'incremental_load.default.orders_target'

In [0]:
#read stage data
stage_df = spark.read.table(stage_table)

In [0]:
stage_df.show()

In [0]:
#check if target table exists
if not spark.catalog.tableExists(target_table_name):
    stage_df.write.format('delta').mode('overwrite').saveAsTable(target_table_name)
else:
    #perform merge query on the target table
    target_table = DeltaTable.forName(spark, target_table_name)

    merge_condition = "s.tracking_num = t.tracking_num"

    target_table.alias('t')\
    .merge(
            stage_df.alias('s'),
            merge_condition
    )\
    .whenMatchedDelete()\
    .execute()

    stage_df.write.format('delta').mode('append').saveAsTable(target_table_name)
       